In [295]:
import numpy as np

In [296]:
# Policy parameterization: Softmax
# State value parameterization: Linear
# State feature vector representation: Fourier (as in HW2)

In [297]:
M = 5 # dimensionality of the fourier transform [2, 10]
# mean_policy_vector = np.ones(2*M+1)*0.05 #some non-zero vector

policy_params = np.ones((2*M+1,2))*0.01
value_params = np.ones(2*M+1)*0.01

softmax_sigma = 0.1
# policy_params = np.zeros((np.power(M+1,2),2))
# value_params = np.zeros(np.power(M+1,2))

policy_step_size = 1e-3 #2e-9
valueFunction_step_size = 1e-3
gamma = 1

In [298]:
def phi_s_cosine(given_x, given_v, cosineFlag):
    normal_x = normalized_x(given_x, cosineFlag)
    normal_v = normalized_v(given_v, cosineFlag)
    feature_phi = np.zeros(2*M+1)
#     feature_phi = np.zeros(np.power(M+1,2))
    feature_phi[0] = 1
    if cosineFlag:
        for i in range(1, M+1):
            feature_phi[i] = np.cos(i*np.pi*normal_x)
        
        for i in range(1, M+1):
            feature_phi[M+i] = np.cos(i*np.pi*normal_v)

#         for i in range(0,M+1):
#             for j in range(0,M+1):
#                 if i==0 or j==0:
#                     feature_phi[(M+1)*i + j] = np.cos(np.pi*(i*normal_x + j*normal_v))
#                 else:
#                     feature_phi[(M+1)*i + j] = np.cos(np.pi*(i*normal_x + j*normal_v))
    
    else:
        for i in range(1, M+1):
            feature_phi[i] = np.sin(i*np.pi*normal_x)
        
        for i in range(1, M+1):
            feature_phi[M+i] = np.sin(i*np.pi*normal_v)

#         for i in range(0,M+1):
#             for j in range(0,M+1):
#                 if i==0 or j==0:
#                     feature_phi[(M+1)*i + j] = np.cos(np.pi*(i*normal_x + j*normal_v))
#                 else:
#                     feature_phi[(M+1)*i + j] = np.cos(np.pi*(i*normal_x + j*normal_v))

    return feature_phi

In [299]:
def normalized_x(given_x, cosineFlag):
    if cosineFlag:
        return (given_x+1.2)/(0.5+1.2)
    else:
        return ((given_x+1.2)/(0.5+1.2))*2 - 1
def normalized_v(given_v, cosineFlag):
    if cosineFlag:
        return (given_v+0.7)/(0.7+0.7)
    else:
        return ((given_v+0.7)/(0.7+0.7))*2 - 1

In [300]:
def runEpisode(policy_params,value_params, gamma):
    
    episode_trace = []
    
    total_reward = 0
    x_0 = np.random.uniform(-0.6,-0.4,1)[0]
    begin_state = (x_0,0)
    
    S_t = begin_state
    S_t_plus_1 = begin_state
    for t in range(0, 1000):
        
        state_t_feature_vector = phi_s_cosine(S_t[0], S_t[1], True)
        state_t_feature_policy_param_product = np.dot(policy_params.T, state_t_feature_vector)
        exp_terms = np.exp(softmax_sigma * state_t_feature_policy_param_product)
        pi_values = exp_terms/np.sum(exp_terms)
        
#         A_t = np.random.choice([-1,1], p=list(pi_values))
        
        max_action_idx = np.argmax(pi_values)
        if max_action_idx==1:
            A_t = 1
        else:
            A_t = -1
        
#         threshold = np.dot(state_feature_vector, given_theta)
        
#         print(threshold)
        
#         if threshold<=0:
#             A_t = -1
#         else:
#             A_t = 1
        
        v_t_plus_1 = S_t[1] + 0.001*A_t - 0.0025*np.cos(3*S_t[0])
        x_t_plus_1 = S_t[0] + v_t_plus_1
        
        
        
        if x_t_plus_1 < -1.2:
            x_t_plus_1 = -1.2
            v_t_plus_1 = 0
        elif x_t_plus_1 > 0.5:
            x_t_plus_1 = 0.5
            v_t_plus_1 = 0
        
        if v_t_plus_1 < -0.7:
            v_t_plus_1 = -0.7
        elif v_t_plus_1 > 0.7:
            v_t_plus_1 = 0.7
        
        S_t_plus_1 = (x_t_plus_1, v_t_plus_1)
        
        current_reward = -1
        if x_t_plus_1 == 0.5:
            current_reward = 0
            print("Terminated")
            episode_trace.append((S_t, A_t, 0))
#             break
        else:
            episode_trace.append((S_t, A_t, -1))

            current_reward = -1

            total_reward += -1
#         print("After ",total_reward)
        
    
        v_hat_s = np.dot(state_t_feature_vector, value_params)
        v_hat_s_prime = np.dot(phi_s_cosine(S_t_plus_1[0], S_t_plus_1[1], True),value_params)
        if x_t_plus_1 == 0.5:
            v_hat_s_prime = 0
        
        delta = current_reward + gamma*v_hat_s_prime - v_hat_s
        
        value_params += valueFunction_step_size*delta*state_t_feature_vector
        

        state_t_feature_policy_param_product = np.dot(policy_params.T, state_t_feature_vector)
        
        
        exp_terms = np.exp(softmax_sigma*state_t_feature_policy_param_product)
        pi_values = exp_terms/np.sum(exp_terms)
        
#         print("Pi values: ",pi_values)
#         print("G: ",G,". Delta: ",delta)
        if A_t == -1:
            policy_params[:,0] += policy_step_size*delta*(1-pi_values[0])*state_t_feature_vector
            policy_params[:,1] += policy_step_size*delta*(-pi_values[0])*state_t_feature_vector
        else:
            policy_params[:,0] += policy_step_size*delta*(-pi_values[1])*state_t_feature_vector
            policy_params[:,1] += policy_step_size*delta*(1-pi_values[1])*state_t_feature_vector
    
    
        if x_t_plus_1 == 0.5:
            break

        S_t = S_t_plus_1 #Incrementing time step
    return episode_trace

In [301]:
#With baseline

curr_iter = 0
while(curr_iter<=5000):
    curr_iter += 1
    policy_params_temp = policy_params.copy()    
    episode_trace = runEpisode(policy_params,value_params, gamma)
    print("\n EPISODE LENGTH: ",len(episode_trace), "CURR ITER: ", curr_iter)

#     for a_step in range(len(episode_trace)):
#         G = 0
#         for idx in range(a_step, len(episode_trace)):
#             G += episode_trace[idx][2]
        
#         state_t = episode_trace[a_step][0]
#         state_t_feature_vector = phi_s_cosine(state_t[0], state_t[1], True)

#         v_hat_s = np.dot(state_t_feature_vector, value_params)
#         delta = G - v_hat_s
        
#         value_params += valueFunction_step_size*delta*state_t_feature_vector
        

#         state_t_feature_policy_param_product = np.dot(policy_params.T, state_t_feature_vector)
        
        
#         exp_terms = np.exp(softmax_sigma*state_t_feature_policy_param_product)
#         pi_values = exp_terms/np.sum(exp_terms)
        
# #         print("Pi values: ",pi_values)
# #         print("G: ",G,". Delta: ",delta)
#         if episode_trace[a_step][1] == -1:
#             policy_params[:,0] += policy_step_size*delta*(1-pi_values[0])*state_t_feature_vector
#             policy_params[:,1] += policy_step_size*delta*(-pi_values[0])*state_t_feature_vector
#         else:
#             policy_params[:,0] += policy_step_size*delta*(-pi_values[1])*state_t_feature_vector
#             policy_params[:,1] += policy_step_size*delta*(1-pi_values[1])*state_t_feature_vector
    
    max_diff = np.max(np.abs(policy_params_temp - policy_params))
    print(policy_step_size," Max diff: ",max_diff)
    if max_diff<policy_step_size*1e-1 or len(episode_trace)<110:# 0.001 works with 1e-6 policy_step
        break


 EPISODE LENGTH:  1000 CURR ITER:  1
0.001  Max diff:  0.0011714248095644562

 EPISODE LENGTH:  1000 CURR ITER:  2
0.001  Max diff:  0.0012913632238950763

 EPISODE LENGTH:  1000 CURR ITER:  3
0.001  Max diff:  0.0028253519324278956

 EPISODE LENGTH:  1000 CURR ITER:  4
0.001  Max diff:  0.0020805278373954295
Terminated

 EPISODE LENGTH:  676 CURR ITER:  5
0.001  Max diff:  0.007810965051052185
Terminated

 EPISODE LENGTH:  697 CURR ITER:  6
0.001  Max diff:  0.006334527713696323
Terminated

 EPISODE LENGTH:  524 CURR ITER:  7
0.001  Max diff:  0.004105583764049441
Terminated

 EPISODE LENGTH:  471 CURR ITER:  8
0.001  Max diff:  0.0033612585493391217
Terminated

 EPISODE LENGTH:  452 CURR ITER:  9
0.001  Max diff:  0.017224959160119246
Terminated

 EPISODE LENGTH:  180 CURR ITER:  10
0.001  Max diff:  0.008296718054411758
Terminated

 EPISODE LENGTH:  225 CURR ITER:  11
0.001  Max diff:  0.007523725387438791
Terminated

 EPISODE LENGTH:  134 CURR ITER:  12
0.001  Max diff:  0.0099773

Terminated

 EPISODE LENGTH:  198 CURR ITER:  103
0.001  Max diff:  0.04954732671935558
Terminated

 EPISODE LENGTH:  203 CURR ITER:  104
0.001  Max diff:  0.047832756028455936
Terminated

 EPISODE LENGTH:  211 CURR ITER:  105
0.001  Max diff:  0.04527701933980177
Terminated

 EPISODE LENGTH:  216 CURR ITER:  106
0.001  Max diff:  0.03514722692084593
Terminated

 EPISODE LENGTH:  208 CURR ITER:  107
0.001  Max diff:  0.04502287567053065
Terminated

 EPISODE LENGTH:  210 CURR ITER:  108
0.001  Max diff:  0.04532901179278248
Terminated

 EPISODE LENGTH:  227 CURR ITER:  109
0.001  Max diff:  0.0675094339638127
Terminated

 EPISODE LENGTH:  285 CURR ITER:  110
0.001  Max diff:  0.0662732061943836
Terminated

 EPISODE LENGTH:  221 CURR ITER:  111
0.001  Max diff:  0.06077424630480677
Terminated

 EPISODE LENGTH:  270 CURR ITER:  112
0.001  Max diff:  0.0657868118393945
Terminated

 EPISODE LENGTH:  216 CURR ITER:  113
0.001  Max diff:  0.046471934711420415
Terminated

 EPISODE LENGTH:  222

Terminated

 EPISODE LENGTH:  332 CURR ITER:  199
0.001  Max diff:  0.12898602936115466
Terminated

 EPISODE LENGTH:  387 CURR ITER:  200
0.001  Max diff:  0.1933501839741334
Terminated

 EPISODE LENGTH:  158 CURR ITER:  201
0.001  Max diff:  0.10791023761152996
Terminated

 EPISODE LENGTH:  240 CURR ITER:  202
0.001  Max diff:  0.10719584518645586
Terminated

 EPISODE LENGTH:  155 CURR ITER:  203
0.001  Max diff:  0.07811241758667187
Terminated

 EPISODE LENGTH:  452 CURR ITER:  204
0.001  Max diff:  0.1627475602762356
Terminated

 EPISODE LENGTH:  389 CURR ITER:  205
0.001  Max diff:  0.17536750623095765
Terminated

 EPISODE LENGTH:  538 CURR ITER:  206
0.001  Max diff:  0.18665369383464814
Terminated

 EPISODE LENGTH:  305 CURR ITER:  207
0.001  Max diff:  0.17313737183202438
Terminated

 EPISODE LENGTH:  379 CURR ITER:  208
0.001  Max diff:  0.12096733758321232
Terminated

 EPISODE LENGTH:  306 CURR ITER:  209
0.001  Max diff:  0.09267496484351057
Terminated

 EPISODE LENGTH:  322 